<a href="https://colab.research.google.com/github/Monika-itds/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install hyperopt

In [13]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard 

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle ('data/train.p')
test = pd.read_pickle ('data/test.p')

X_train, y_train = train ['features'], train ['labels']
X_test, y_test = test ['features'], test ['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape= X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
    model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'])
    
    logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S')) #sciezka, w ktorej trzymane beda logi
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)

    model.fit(
        X_train, 
        y_train,
        batch_size=params_fit.get('batch_size', 128),
        epochs=params_fit.get('epochs', 5),
        verbose=params_fit.get ('verbose', 1),
        validation_data=params_fit.get('validation_data', (X_train, y_train)),
        callbacks = [tensorboard_callback]
    )

    return model

def predict (model_trained, X_test, y_test, scoring=accuracy_score):
  
  y_test_norm = np.argmax(y_test, axis = 1)
  
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis = 1)
  
  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32,kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32,kernel_size=(3,3), activation='relu', padding='same'), 
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64,kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64,kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64,kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64,kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),  

    Dense(1024, activation='relu'),
    Dropout(0.3), #Dropuot()-jest po to, aby model nie przeuczył się za szybko

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),
    ])


In [23]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model,X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 6s 23ms/step - loss: 2.5877 - accuracy: 0.2824 - val_loss: 1.0561 - val_accuracy: 0.6565
Epoch 2/5
272/272 [==============================] - 6s 22ms/step - loss: 0.9510 - accuracy: 0.6940 - val_loss: 0.2442 - val_accuracy: 0.9284
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.4879 - accuracy: 0.8486 - val_loss: 0.1089 - val_accuracy: 0.9674
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2959 - accuracy: 0.9095 - val_loss: 0.0625 - val_accuracy: 0.9818
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2263 - accuracy: 0.9333 - val_loss: 0.0485 - val_accuracy: 0.9864


0.9616780045351474

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32,kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32,kernel_size=(3,3), activation='relu', padding='same'), 
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64,kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64,kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128,kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128,kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),  

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),
    ])


In [0]:
def func_obj (params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics=['accuracy'])
  
  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
  )

  score = model.evaluate(X_test,y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss':-accuracy, 'status': STATUS_OK, 'model':model}


In [29]:
#optymalizacja parametrow dropout i batch_size, tj.wybranie przestrzeni dla modelu, w ktorej moze sprawdzac poszczegolne wartosci parametrow
space = {
    'batch_size':hp.quniform('batch_size', 100, 200,10),
    'dropout_cnn_block_one':hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two':hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three':hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one':hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two':hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,#algorytm, ktory uzywamy
    30,#ilisc wykonywanych prob
    Trials()#tu beda zapisywane wyniki

)

{'batch_size': 100.0, 'dropout_cnn_block_one': 0.3030487311413978, 'dropout_cnn_block_three': 0.45297140452096285, 'dropout_cnn_block_two': 0.3300768606237274, 'dropout_dense_block_one': 0.5352344645162823, 'dropout_dense_block_two': 0.4314272169921096}
accuracy=0.9501133561134338
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.47521239671803706, 'dropout_cnn_block_three': 0.32892436782719564, 'dropout_cnn_block_two': 0.34906328890066757, 'dropout_dense_block_one': 0.506118148538518, 'dropout_dense_block_two': 0.5673378219821469}
accuracy=0.880725622177124
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.36976738991849323, 'dropout_cnn_block_three': 0.3465155695363398, 'dropout_cnn_block_two': 0.43398746013522893, 'dropout_dense_block_one': 0.6644494858015565, 'dropout_dense_block_two': 0.6583275947353778}
accuracy=0.8925170302391052
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.4982272135497968, 'dropout_cnn_block_three': 0.49200321639918365, 'dropout_cnn_block_two': 0.38844080474